In [3]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as plt
from sklearn import linear_model

In [132]:
data = pd.read_csv("Pokemon 2.csv")
data.head()
print(type(data.iloc[4,3]))

<class 'float'>


In [143]:
#Train the Machine Learning model with the sample data
#Create a 2D array of values detailing the average of each stat for each typing
model = pd.DataFrame()
model['Type'] = ['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying']
stats = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed','Total_w_sec', 'HP_w_sec', 'Attack_w_sec', 'Defense_w_sec', 'Sp. Atk_w_sec', 'Sp. Def_w_sec', 'Speed_w_sec', 'Total_Num_Primary', 'Total_Num_Secondary']


for column in stats:
    model[column] = 0

num_per_type = data['Type 1'].value_counts()
num_per_type_2 = data['Type 2'].value_counts()

for i in num_per_type.index:
    m = model.loc[model['Type'] == i] 
    model.loc[m.index, 'Total_Num_Primary'] = num_per_type[i]


for i in num_per_type_2.index:
    m = model.loc[model['Type'] == i]
    model.loc[m.index, 'Total_Num_Secondary'] = num_per_type_2[i] + model.loc[m.index, 'Total_Num_Primary']

for index1, typing in model.iterrows():
    for index2, pokemon in data.iterrows():
        if pokemon['Type 1'] == typing['Type']:
            for index in range(4,11):
                model.loc[index1, data.columns[index]] += pokemon[index]
                model.loc[index1, data.columns[index] + '_w_sec'] += pokemon[index]
        if pokemon['Type 2'] and pokemon['Type 2'] == typing['Type']:
            for index in range(4,11):
                model.loc[index1, data.columns[index] + '_w_sec'] += pokemon[index]
    for col_index in range(1,8):
        model.loc[index1, model.columns[col_index]] /= model.loc[index1,'Total_Num_Primary']
        model.loc[index1, model.columns[col_index+7]] /= model.loc[index1,'Total_Num_Secondary']
#normalize values around 1 to account for variability between weak/strong pokemon


    for col_index in range(2,8):
        model.loc[index1, model.columns[col_index]] /= (model.loc[index1,'Total']/6)
        model.loc[index1, model.columns[col_index+7]] /= (model.loc[index1,'Total_w_sec']/6)


model.head()

,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total_w_sec,HP_w_sec,Attack_w_sec,Defense_w_sec,Sp. Atk_w_sec,Sp. Def_w_sec,Speed_w_sec,Total_Num_Primary,Total_Num_Secondary
0,Grass,421.142857,0.958412,1.043080,1.008684,1.104138,1.003392,0.882293,417.926316,0.948291,1.054681,1.051659,1.047125,1.026572,0.871672,70,95
1,Fire,458.076923,0.915617,1.110327,0.887657,1.165491,0.945844,0.975063,467.109375,0.901154,1.080381,0.900351,1.194581,0.954742,0.968791,52,64
2,Water,430.455357,1.004460,1.033582,1.016780,1.042791,0.982929,0.919458,429.095238,0.993120,1.030629,1.038841,1.045611,0.983132,0.908667,112,126
3,Bug,378.927536,0.900711,1.123767,1.119865,0.852979,1.026008,0.976669,379.527778,0.896948,1.134524,1.124204,0.846886,1.022323,0.975115,69,72
4,Normal,401.683673,1.154274,1.097422,0.893941,0.833736,0.951861,1.068767,402.068627,1.145107,1.084246,0.889517,0.851771,0.951257,1.078101,98,102


In [201]:
pok_ind = 7
pok = data.iloc[pok_ind, 4:11]
arr = np.empty(18)
resid = 0
for col in range (1,7):
    pok[col] = (6*pok[col])/pok[0]
arr = np.empty(18)
resid = 0
for index, Type in model.iterrows():
    for col in range(1,7):
        resid += abs(pok[col]-model.iloc[index, col+1])
    # if pd.to_numeric(po).idxmax() == pd.to_numeric(Type[2:8]).idxmax():
    #     resid -= .05
    arr[index] = resid
    resid = 0
result_ind = np.where(arr == np.amin(arr))
po = pok[1:7]


In [228]:
#use the predictive model to guess a pokemon's typing based on its stats alone

def predict(pok):
    #normalize the stats around 1
    for col in range (1,7):
        pok[col] = (6*pok[col])/pok[0]
    arr = np.empty(18)
    resid = 0
    for index, Type in model.iterrows():
        for col in range(1,7):
            resid += abs(pok[col]-model.iloc[index, col+1])
        arr[index] = resid + 1/model.loc[index, 'Total_Num_Primary']
        resid = 0
    result_ind = np.where(arr == np.amin(arr))

    return(model.loc[result_ind[0][0], 'Type'])

def predict_w_sec(pok):
    #normalize the stats around 1
    for col in range (1,7):
        pok[col] = (6*pok[col])/pok[0]
    arr = np.empty(18)
    resid = 0
    for index, Type in model.iterrows():
        for col in range(1,7):
            resid += abs(pok[col]-model.iloc[index, col+8])
        arr[index] = resid
        resid = 0
    result_ind = np.where(arr == np.amin(arr))
    return(model.loc[result_ind[0][0], 'Type'])


In [203]:
for col in range(1,7):
    print(model.iloc[1,col+8])

    

0.9011540391369794
1.0803813346713498
0.900351229302559
1.1945810336176619
0.954741595584546
0.9687907676869042


In [229]:
#Analysis of algorithms
count = 0
count_sec = 0
model['Guessed_Correct'] = 0
model['Guessed_Correct_sec'] = 0
model['Guessed_in_Error'] = 0
model['Guessed_in_Error_sec'] = 0
for index, pok in data.iterrows():
    guess = predict(pok[4:11])
    guess_sec = predict_w_sec(pok[4:11])
    #  or guess == pok[3]:

    if guess == pok[2]:
        count+=1
        model.loc[model['Type'] == guess, 'Guessed_Correct'] += 1
    else:
        model.loc[model['Type'] == guess, 'Guessed_in_Error'] +=1
    if guess_sec == pok[2]:
        count_sec+=1
        model.loc[model['Type'] == guess_sec, 'Guessed_Correct_sec'] += 1
    else:
        model.loc[model['Type'] == guess_sec, 'Guessed_in_Error_sec'] +=1

print(count,count_sec)
print(model.loc[model['Type']=='Steel'])

124 57
     Type       Total   HP    Attack   Defense  Sp. Atk  Sp. Def    Speed  \
16  Steel  487.703704  1.0  1.140492  1.554678  0.83065  0.99195  0.67983   

    Total_w_sec  HP_w_sec  ...  Defense_w_sec  Sp. Atk_w_sec  Sp. Def_w_sec  \
16   486.591837   0.80099  ...       1.437906       0.889066       1.025458   

    Speed_w_sec  Total_Num_Primary  Total_Num_Secondary  Guessed_Correct  \
16     0.704106                 27                   49               14   

    Guessed_Correct_sec  Guessed_in_Error  Guessed_in_Error_sec  
16                    0                62                     0  

[1 rows x 21 columns]


In [231]:
print((model['Guessed_in_Error'].idxmax()))
print(model.loc[6])
model.head()

6
Type                      Electric
Total                   443.409091
HP                        0.809124
Attack                    0.934905
Defense                   0.897078
Sp. Atk                   1.218145
Sp. Def                   0.997335
Speed                     1.143414
Total_w_sec                 444.84
HP_w_sec                  0.852441
Attack_w_sec              0.937685
Defense_w_sec             0.897491
Sp. Atk_w_sec             1.199892
Sp. Def_w_sec             0.993796
Speed_w_sec               1.118694
Total_Num_Primary               44
Total_Num_Secondary             50
Guessed_Correct                 10
Guessed_Correct_sec              3
Guessed_in_Error                71
Guessed_in_Error_sec            33
Name: 6, dtype: object


,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total_w_sec,HP_w_sec,...,Defense_w_sec,Sp. Atk_w_sec,Sp. Def_w_sec,Speed_w_sec,Total_Num_Primary,Total_Num_Secondary,Guessed_Correct,Guessed_Correct_sec,Guessed_in_Error,Guessed_in_Error_sec
0,Grass,421.142857,0.958412,1.043080,1.008684,1.104138,1.003392,0.882293,417.926316,0.948291,...,1.051659,1.047125,1.026572,0.871672,70,95,3,21,7,194
1,Fire,458.076923,0.915617,1.110327,0.887657,1.165491,0.945844,0.975063,467.109375,0.901154,...,0.900351,1.194581,0.954742,0.968791,52,64,12,8,49,72
2,Water,430.455357,1.004460,1.033582,1.016780,1.042791,0.982929,0.919458,429.095238,0.993120,...,1.038841,1.045611,0.983132,0.908667,112,126,6,5,20,40
3,Bug,378.927536,0.900711,1.123767,1.119865,0.852979,1.026008,0.976669,379.527778,0.896948,...,1.124204,0.846886,1.022323,0.975115,69,72,4,1,31,0
4,Normal,401.683673,1.154274,1.097422,0.893941,0.833736,0.951861,1.068767,402.068627,1.145107,...,0.889517,0.851771,0.951257,1.078101,98,102,26,8,31,38


In [190]:
model.loc[model['Type'] == 'Psychic', 'Guessed_in_Error_sec'] +=1
model.head()

,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total_w_sec,HP_w_sec,...,Defense_w_sec,Sp. Atk_w_sec,Sp. Def_w_sec,Speed_w_sec,Total_Num_Primary,Total_Num_Secondary,Guessed_Correct,Guessed_Correct_sec,Guessed_in_Error,Guessed_in_Error_sec
0,Grass,421.142857,0.958412,1.043080,1.008684,1.104138,1.003392,0.882293,417.926316,0.948291,...,1.051659,1.047125,1.026572,0.871672,70,95,0,0,0,0
1,Fire,458.076923,0.915617,1.110327,0.887657,1.165491,0.945844,0.975063,467.109375,0.901154,...,0.900351,1.194581,0.954742,0.968791,52,64,0,0,0,0
2,Water,430.455357,1.004460,1.033582,1.016780,1.042791,0.982929,0.919458,429.095238,0.993120,...,1.038841,1.045611,0.983132,0.908667,112,126,0,0,0,0
3,Bug,378.927536,0.900711,1.123767,1.119865,0.852979,1.026008,0.976669,379.527778,0.896948,...,1.124204,0.846886,1.022323,0.975115,69,72,0,0,0,0
4,Normal,401.683673,1.154274,1.097422,0.893941,0.833736,0.951861,1.068767,402.068627,1.145107,...,0.889517,0.851771,0.951257,1.078101,98,102,0,0,0,0
